## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Wajid,SO,3.81,43.25,75.16,82,85,13.56,overcast clouds
1,1,Chokurdakh,RU,70.63,147.92,10.85,93,97,7.00,overcast clouds
2,2,Carnarvon,AU,-24.87,113.63,77.00,57,0,19.46,clear sky
3,3,Cherskiy,RU,68.75,161.30,20.25,94,5,5.39,clear sky
4,4,Barrow,US,71.29,-156.79,28.40,86,90,9.17,fog


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Wajid,SO,3.81,43.25,75.16,82,85,13.56,overcast clouds
2,2,Carnarvon,AU,-24.87,113.63,77.00,57,0,19.46,clear sky
5,5,Busselton,AU,-33.65,115.33,73.99,30,0,6.91,clear sky
9,9,Luanda,AO,-8.84,13.23,73.40,88,75,8.05,broken clouds
10,10,Biak,ID,-0.91,122.88,85.87,64,13,3.20,few clouds
11,11,Butaritari,KI,3.07,172.79,82.18,71,7,10.83,clear sky
12,12,Yanam,IN,16.73,82.22,78.80,94,20,3.36,mist
15,15,Mataura,NZ,-46.19,168.86,70.00,51,57,10.00,broken clouds
19,19,Rikitea,PF,-23.12,-134.97,73.96,83,100,16.04,overcast clouds
20,20,Hobyo,SO,5.35,48.53,80.46,78,100,5.30,moderate rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                249
City                   249
Country                249
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Current Description    249
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Wajid,SO,3.81,43.25,75.16,82,85,13.56,overcast clouds
2,2,Carnarvon,AU,-24.87,113.63,77.00,57,0,19.46,clear sky
5,5,Busselton,AU,-33.65,115.33,73.99,30,0,6.91,clear sky
9,9,Luanda,AO,-8.84,13.23,73.40,88,75,8.05,broken clouds
10,10,Biak,ID,-0.91,122.88,85.87,64,13,3.20,few clouds
...,...,...,...,...,...,...,...,...,...,...
663,663,San Andres,CO,12.58,-81.70,80.60,88,20,2.48,few clouds
664,664,Camocim,BR,-2.90,-40.84,77.34,86,10,12.01,clear sky
668,668,Belmonte,BR,-15.86,-38.88,71.60,94,90,2.24,overcast clouds
669,669,Caravelas,BR,-17.71,-39.25,72.27,92,51,6.17,light rain


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Wajid,SO,75.16,overcast clouds,3.81,43.25,
2,Carnarvon,AU,77.00,clear sky,-24.87,113.63,
5,Busselton,AU,73.99,clear sky,-33.65,115.33,
9,Luanda,AO,73.40,broken clouds,-8.84,13.23,
10,Biak,ID,85.87,few clouds,-0.91,122.88,
11,Butaritari,KI,82.18,clear sky,3.07,172.79,
12,Yanam,IN,78.80,mist,16.73,82.22,
15,Mataura,NZ,70.00,broken clouds,-46.19,168.86,
19,Rikitea,PF,73.96,overcast clouds,-23.12,-134.97,
20,Hobyo,SO,80.46,moderate rain,5.35,48.53,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [ ]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

In [ ]:
hotel_df.head(10)

In [ ]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


In [ ]:
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]